In [2]:
import pandas as pd
import os
import torch
#import torchaudio
#import requests
#import matplotlib
#import matplotlib.pyplot as plt
#import IPython
#import gc
import numpy as np

from os.path import isfile, join
from scipy.io.wavfile import write, read
from scipy.stats import kurtosis, skew
#from datasets import load_dataset
from tqdm import tqdm
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model, Wav2Vec2ForPreTraining, Wav2Vec2ForXVector
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices

In [3]:
#home_path = '/home/hpc/iwi5/iwi5055h/'
home_path = ''
icp_path = home_path + "data/IEMOCAP/"
save_path = icp_path + "wav2vec/"

In [4]:
#model_mode = 'base'
model_mode = 'xlsr'
#model_mode = 'xvec'
# 'xlsr' | 'base' | 'xvec'

In [5]:
#device = 'cpu'
#if torch.cuda.is_available():
 #   device = 'cuda'

In [6]:
def Wav2Vec_Models(file, model_name='xvec', plot=False, cuda=False):

    try:
        fs, audio=read(file)   
        audio=(audio - np.mean(audio))/max(audio)
       
        device='cpu'
        if cuda:
            device='cuda'
       
        #sampling_rate = fs
       
        if model_name=='xlsr':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
        elif model_name=='base':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")   
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base-960h").to(device)
        elif model_name=='xvec':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/wav2vec2-base-superb-sv")        
            #feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-er")        
            model = Wav2Vec2ForXVector.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
   
        # audio file is decoded on the fly
        input_values = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True).to(device).input_values
        input_values_all = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True)

        # compute masked indices
        batch_size, raw_sequence_length = input_values.shape
        sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length)
        mask_time_indices = _compute_mask_indices((batch_size, sequence_length), mask_prob=0.2, mask_length=2)
        mask_time_indices = torch.tensor(mask_time_indices, device=input_values.device, dtype=torch.long)

        with torch.inference_mode():

            tmp = model(**input_values_all.to(device))
            #tmp_proj=model_proj(input_values.to(device), mask_time_indices=mask_time_indices.to(device) )
            if model_name=='xvec':
                last_hidden_states = tmp.embeddings.cpu()
                #last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                features = tmp.logits.cpu()
                #features = np.hstack((features,features_std,features_skew,features_kurt,features_min,features_max))

                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

            else:
                last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())
                last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())
                last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)
                last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)
                last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())
                last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())

                last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())
                features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())
                features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)
                features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)
                features_min=np.array(torch.min(tmp.extract_features, dim=1)[0].cpu())
                features_max=np.array(torch.max(tmp.extract_features, dim=1)[0].cpu())

                features = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

        return features[0], last_hidden_states[0]
    except:
        print('sequence not long enough')


### Testing a file

In [6]:
#test_file = 'data/AIBO/wav/Mont_01_053_00.wav'

In [8]:
#model_name = 'base'
#device = 'cpu'
#if torch.cuda.is_available():
 #   device = 'cuda'

In [9]:
#fs, audio=read(test_file)   
#audio=(audio-np.mean(audio))/max(audio)
#sampling_rate = fs

#if model_name=='xlsr':
 #   feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
  #  model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
   # #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
#elif model_name=='base':
 #   feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")   
  #  model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
   # #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base-960h").to(device)
#elif model_name=='xvec':
 #   feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/wav2vec2-base-superb-sv")        
  #  model = Wav2Vec2ForXVector.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
    #feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-er")  # size mistmatch        
    #model = Wav2Vec2ForXVector.from_pretrained("superb/wav2vec2-base-superb-er").to(device)
    #model_proj = Wav2Vec2ForPreTraining.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)

In [10]:
# audio file is decoded on the fly
#input_values = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True).to(device).input_values
#input_values_all = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True)

# compute masked indices
#batch_size, raw_sequence_length = input_values.shape
#sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length)
#mask_time_indices = _compute_mask_indices((batch_size, sequence_length), mask_prob=0.2, mask_length=2)
#mask_time_indices = torch.tensor(mask_time_indices, device=input_values.device, dtype=torch.long)

#with torch.inference_mode():
           
 #   tmp = model(**input_values_all.to(device))
    #tmp_proj=model_proj(input_values.to(device), mask_time_indices=mask_time_indices.to(device) )
  #  if model_name=='xvec':
   #     last_hidden_states = tmp.embeddings.cpu()
        #last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

    #    features = tmp.logits.cpu()
        #features = np.hstack((features,features_std,features_skew,features_kurt,features_min,features_max))

        #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

        #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))
'''
    else:
        last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())
        last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())
        last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)
        last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)
        last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())
        last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())

        last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

        features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())
        features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())
        features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)
        features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)
        features_min=np.array(torch.min(tmp.extract_features, dim=1)[0].cpu())
        features_max=np.array(torch.max(tmp.extract_features, dim=1)[0].cpu())

        features = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

        #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

        #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

print(features[0], last_hidden_states[0])
'''

'\n    else:\n        last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())\n        last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())\n        last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)\n        last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)\n        last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())\n        last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())\n\n        last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))\n\n        features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())\n        features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())\n        features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)\n        features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)\n        features_min=np.array(tor

## Creating .csv

In [7]:
full_labels_df = pd.read_csv(icp_path + "metadata.csv")
# Create a column for gender
full_labels_df['gender'] = full_labels_df.ID.str[-1:]
# Convert gender to numeric
full_labels_df['gender'] = full_labels_df['gender'].map(dict(zip(['M', 'F'], [0, 1])))
# emotions = np.array(['ang', 'hap', 'sad', 'neu', 'fru', 'exc', 'fea', 'sur', 'dis'])
# filt_df = labels_df[labels_df['emotion'].isin(emotions)]
#iemocap_list = labels_df[['File Name', 'emotion', 'gender']].copy()  # wav_file, label
full_labels_df['session'] = full_labels_df['ID'].str[1:2]
full_labels_df['path'] = icp_path + "Session" + full_labels_df['session'] + "/" + full_labels_df['File Name'] + ".wav"

In [8]:
#USED_CLASSES = ["neu", "hap", "sad", "ang", "fru", "exc"]
USED_CLASSES = ["neu", "hap", "sad", "fru", "ang"]
CLASS_TO_ID = {"neu": 0, "hap": 1, "sad": 2, "fru": 3, "ang": 4}

In [9]:
full_labels_df["session"] = pd.to_numeric(full_labels_df["session"])

In [10]:
train_index = [1,3,5]
test_index = 4
val_index = 2

In [11]:
full_labels_df['split']=full_labels_df['session'].apply(lambda x: 'test' if x == test_index else ('val' if x == val_index else 'train'))

In [12]:
# merge "exc" into "hap"
full_labels_df.loc[full_labels_df['emotion'] == 'exc', 'emotion'] = 'hap'

In [13]:
full_labels_df['lb'] = full_labels_df['emotion'].map(CLASS_TO_ID)

In [14]:
filtered_df = full_labels_df[full_labels_df['lb'].isnull()]
filtered_df.emotion.value_counts()

xxx    2507
sur     107
fea      40
oth       3
dis       2
Name: emotion, dtype: int64

In [15]:
labels_df = full_labels_df[full_labels_df['emotion'].isin(USED_CLASSES)]

In [16]:
labels_df[labels_df.split=='test']

,ID,File Name,arousal,dominance,emotion,valence,Quadrant,gender,session,path,split,lb
5767,04F,Ses04F_impro01_F001,2.5,4.0,fru,4.0,0,1,4,data/IEMOCAP/Session4/Ses04F_impro01_F001.wav,test,3.0
5768,04F,Ses04F_impro01_F002,2.0,4.5,fru,4.5,3,1,4,data/IEMOCAP/Session4/Ses04F_impro01_F002.wav,test,3.0
5769,04F,Ses04F_impro01_F003,2.5,4.5,fru,4.0,0,1,4,data/IEMOCAP/Session4/Ses04F_impro01_F003.wav,test,3.0
5770,04F,Ses04F_impro01_F004,2.0,4.5,ang,4.0,3,1,4,data/IEMOCAP/Session4/Ses04F_impro01_F004.wav,test,4.0
5771,04F,Ses04F_impro01_F005,1.5,4.5,fru,4.0,3,1,4,data/IEMOCAP/Session4/Ses04F_impro01_F005.wav,test,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7864,04M,Ses04M_script03_2_M052,2.5,4.0,ang,4.5,0,0,4,data/IEMOCAP/Session4/Ses04M_script03_2_M052.wav,test,4.0
7865,04M,Ses04M_script03_2_M053,1.5,4.0,ang,4.0,3,0,4,data/IEMOCAP/Session4/Ses04M_script03_2_M053.wav,test,4.0
7866,04M,Ses04M_script03_2_M054,1.5,4.0,ang,4.0,3,0,4,data/IEMOCAP/Session4/Ses04M_script03_2_M054.wav,test,4.0
7867,04M,Ses04M_script03_2_M055,1.5,4.0,ang,4.0,3,0,4,data/IEMOCAP/Session4/Ses04M_script03_2_M055.wav,test,4.0


In [17]:
labels_df[labels_df.split=='val']

,ID,File Name,arousal,dominance,emotion,valence,Quadrant,gender,session,path,split,lb
1819,02F,Ses02F_impro01_F000,4.0,2.0,neu,3.0,0,1,2,data/IEMOCAP/Session2/Ses02F_impro01_F000.wav,val,0.0
1822,02F,Ses02F_impro01_F003,2.0,2.5,fru,3.5,3,1,2,data/IEMOCAP/Session2/Ses02F_impro01_F003.wav,val,3.0
1823,02F,Ses02F_impro01_F004,2.0,2.0,fru,3.5,3,1,2,data/IEMOCAP/Session2/Ses02F_impro01_F004.wav,val,3.0
1824,02F,Ses02F_impro01_F005,2.0,3.0,fru,3.5,3,1,2,data/IEMOCAP/Session2/Ses02F_impro01_F005.wav,val,3.0
1825,02F,Ses02F_impro01_F006,2.0,2.5,fru,3.0,3,1,2,data/IEMOCAP/Session2/Ses02F_impro01_F006.wav,val,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3625,02M,Ses02M_script03_2_M043,2.0,4.0,ang,2.5,3,0,2,data/IEMOCAP/Session2/Ses02M_script03_2_M043.wav,val,4.0
3626,02M,Ses02M_script03_2_M044,1.5,4.0,ang,2.5,3,0,2,data/IEMOCAP/Session2/Ses02M_script03_2_M044.wav,val,4.0
3627,02M,Ses02M_script03_2_M045,2.0,3.5,ang,3.5,3,0,2,data/IEMOCAP/Session2/Ses02M_script03_2_M045.wav,val,4.0
3628,02M,Ses02M_script03_2_M046,2.5,4.0,ang,3.5,0,0,2,data/IEMOCAP/Session2/Ses02M_script03_2_M046.wav,val,4.0


In [18]:
labels_df.to_csv(icp_path + 'final_labels_df.csv')

## Extracting .npy samplewise

In [21]:
labels_df = pd.read_csv(icp_path + 'final_labels_df.csv', header=0, index_col=0)
labels_df

,ID,File Name,arousal,dominance,emotion,valence,Quadrant,gender,session,path,split,lb
0,01F,Ses01F_impro01_F000,2.5,2.5,neu,2.5,0,1,1,data/IEMOCAP/Session1/Ses01F_impro01_F000.wav,train,0.0
1,01F,Ses01F_impro01_F001,2.5,2.5,neu,2.5,0,1,1,data/IEMOCAP/Session1/Ses01F_impro01_F001.wav,train,0.0
2,01F,Ses01F_impro01_F002,2.5,2.5,neu,2.5,0,1,1,data/IEMOCAP/Session1/Ses01F_impro01_F002.wav,train,0.0
5,01F,Ses01F_impro01_F005,2.5,2.0,neu,3.5,0,1,1,data/IEMOCAP/Session1/Ses01F_impro01_F005.wav,train,0.0
6,01F,Ses01F_impro01_F006,2.0,3.5,fru,3.5,3,1,1,data/IEMOCAP/Session1/Ses01F_impro01_F006.wav,train,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10034,05M,Ses05M_script03_2_M041,1.0,5.0,ang,4.5,3,0,5,data/IEMOCAP/Session5/Ses05M_script03_2_M041.wav,train,4.0
10035,05M,Ses05M_script03_2_M042,1.0,4.5,ang,4.5,3,0,5,data/IEMOCAP/Session5/Ses05M_script03_2_M042.wav,train,4.0
10036,05M,Ses05M_script03_2_M043,1.5,4.5,ang,4.0,3,0,5,data/IEMOCAP/Session5/Ses05M_script03_2_M043.wav,train,4.0
10037,05M,Ses05M_script03_2_M044,1.0,5.0,ang,5.0,3,0,5,data/IEMOCAP/Session5/Ses05M_script03_2_M044.wav,train,4.0


In [20]:
def w2v_path(file, split):    
    return save_path + model_mode + "_extract_features/" + split + '-' + file + '.npy'

In [21]:
na_files = []
problems = []

In [22]:
for k,v in labels_df.iterrows():
    try:
        if not isfile(v['path']):
            na_files.append(v['path'])
            print("wav for {} not available". format(v['File Name']))
        elif isfile(w2v_path(v['File Name'], v['split'])):
            pass
            #print("w2v features found for {}".format(v['File Name']))
        else:
            #ft, hs = Wav2Vec_Model(v['path'])
            ft, hs = Wav2Vec_Models(v['path'], model_name=model_mode, cuda=True)
            #np.save(save_path + "extract_features/" + v['split'] + '-' + v['filename'] + '.npy', ft)
            np.save(save_path +  model_mode + "_extract_features/" + v['split'] + '-' + v['File Name'] + '.npy', ft)
            np.save(save_path +  model_mode + "_hidden_states/" + v['split'] + '-' + v['File Name'] + '.npy', hs)
            print("saved for {}".format(v['File Name']))
    except:
        problems.append(v['File Name'])
        print("wrong")

In [23]:
na_files

[]

In [24]:
problems

[]

## Gathering .npy arrays into train/val/test with features, hidden states and layers

In [25]:
icp_ft_train = []
icp_ft_val = []
icp_ft_test = []

In [26]:
icp_hs_train = []
icp_hs_val = []
icp_hs_test = []

In [27]:
icp_lb_train = []
icp_lb_val = []
icp_lb_test = []

In [28]:
for file in os.listdir(save_path + model_mode + "_extract_features/"):
    if file.startswith("train-"):
        #filename = file.split("-")[1]
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        icp_ft_train.append(ft)
    elif file.startswith("val-"):
        #filename = file.split("-")[1]
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        icp_ft_val.append(ft)
    else:
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        icp_ft_test.append(ft)        
np.save(save_path + model_mode + "_icp_ft_train.npy", icp_ft_train)
np.save(save_path + model_mode + "_icp_ft_val.npy", icp_ft_val)
np.save(save_path + model_mode + "_icp_ft_test.npy", icp_ft_test)

In [29]:
for file in os.listdir(save_path + model_mode + "_hidden_states/"):
    if file.startswith("train-"):
        #filename = file.split("-")[1]
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        icp_hs_train.append(hs)
    elif file.startswith("val-"):
        #filename = file.split("-")[1]
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        icp_hs_val.append(hs)
    else:
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        icp_hs_test.append(hs)        
np.save(save_path + model_mode + "_icp_hs_train.npy", icp_hs_train)
np.save(save_path + model_mode + "_icp_hs_val.npy", icp_hs_val)
np.save(save_path + model_mode + "_icp_hs_test.npy", icp_hs_test)

In [30]:
labels_df.dtypes

ID            object
File Name     object
arousal      float64
dominance    float64
emotion       object
valence      float64
Quadrant       int64
gender         int64
session        int64
path          object
split         object
lb           float64
dtype: object

In [31]:
for file in os.listdir(save_path + model_mode + "_extract_features/"):
    label = labels_df.loc[labels_df['File Name'] == file.split("-")[1].split(".")[0], 'lb']
    lb = int(label.tolist()[0])
    if file.startswith("train-"):
        icp_lb_train.append(lb)
    elif file.startswith("val-"):
        icp_lb_val.append(lb)
    else:
        icp_lb_test.append(lb)
np.save(save_path + model_mode + "_icp_lb_train.npy", icp_lb_train)
np.save(save_path + model_mode + "_icp_lb_val.npy", icp_lb_val)
np.save(save_path + model_mode + "_icp_lb_test.npy", icp_lb_test)

In [32]:
#len(os.listdir(save_path + model_mode + "_extract_features/"))

In [33]:
len(icp_lb_val)

1348

In [34]:
len(icp_lb_train)

4520

In [35]:
#len(aibo_hs_val)

In [36]:
len(icp_lb_test)

1512

In [37]:
#len(aibo_hs_test)

In [38]:
#aibo_lb_train[634]

In [26]:
#type(aibo_lb_test[0].replace)

In [27]:
#len(aibo_lb_val)

84

In [118]:
#str(aibo_lb_val[0]).replace('\n', ' ').split(' ')

['2']

In [137]:
#for i in range(len(aibo_lb_val)):
 #   a = str(aibo_lb_val[i]).replace('\n', ' ').split(' ')[4]
  #  aibo_lb_val[i] = int(float(a))

In [138]:
#type(int(float(aibo_lb_test[0])))

int

In [31]:
#len(aibo_lb_train[2].split(' '))

8

In [40]:
#int(float(aibo_lb_train[2].replace('\n', ' ').split(' ')[4]))

1

In [39]:
#type(int(float(aibo_lb_train[2].replace('\n', ' ').split(' ')[4])))

int

In [ ]:
#np.unique(labels)

In [17]:
#len(os.listdir('data/AIBO/wav/'))

3547

In [21]:
#len(na_files)

14874